# GraphRAG风格的医学知识图谱实体关系抽取

本notebook参考GraphRAG的思想，使用LLM从医学文档中自动提取实体和关系。

## Schema定义

**实体类型：**
- Disease（疾病）
- Symptom（症状）
- Test（检查方法）
- Treatment（治疗方法）
- Pathogen（病原体）
- RiskFactor（风险因素）
- DifferentialDiagnosis（鉴别诊断）
- LiteratureSource（文献来源）

**关系类型：**
- HAS_SYMPTOM（疾病-症状）
- DIAGNOSED_BY（疾病-检查方法）
- TREATED_WITH（疾病-治疗方法）
- CAUSED_BY（疾病-病原体）
- HAS_RISK_FACTOR（疾病-风险因素）
- DIFFERENTIAL_DIAGNOSIS（疾病-鉴别诊断）
- SOURCE_FROM（实体-文献来源）


## 1. 环境设置和导入依赖


In [54]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict, Optional
import json
from pathlib import Path
import time

print("依赖导入成功")


依赖导入成功


## 2. 配置LLM


In [55]:
# 配置LLM
llm = ChatOpenAI(
    model="qwen2.5:14b",
    base_url="https://zjlchat.vip.cpolar.cn/v1",
    api_key="EMPTY",
    temperature=0.1,
    top_p=0.8
)

print("LLM配置完成")


LLM配置完成


## 3. 定义数据结构


In [56]:
# 定义实体结构
class Entity(BaseModel):
    """实体定义"""
    name: str = Field(description="实体名称")
    entity_type: str = Field(description="实体类型：Disease/Symptom/Test/Treatment/Pathogen/RiskFactor/DifferentialDiagnosis/LiteratureSource")
    description: str = Field(description="实体描述")

class Relationship(BaseModel):
    """关系定义"""
    source: str = Field(description="源实体名称")
    target: str = Field(description="目标实体名称")
    relation_type: str = Field(description="关系类型：HAS_SYMPTOM/DIAGNOSED_BY/TREATED_WITH/CAUSED_BY/HAS_RISK_FACTOR/DIFFERENTIAL_DIAGNOSIS/SOURCE_FROM")
    description: Optional[str] = Field(default="", description="关系描述")

class KnowledgeGraph(BaseModel):
    """知识图谱结构"""
    entities: List[Entity] = Field(description="实体列表")
    relationships: List[Relationship] = Field(description="关系列表")

print("数据结构定义完成")


数据结构定义完成


## 4. 创建提取Prompt


In [57]:
# 创建实体关系抽取的提示模板
extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个专业的医学知识图谱构建专家。你的任务是从医学文本中提取结构化的实体和关系。

**实体类型定义：**
- Disease（疾病）：各种疾病名称，包括主要疾病和特殊类型
- Symptom（症状）：临床症状和体征
- Test（检查方法）：诊断检查和评分系统
- Treatment（治疗方法）：治疗手段和药物
- Pathogen（病原体）：致病微生物
- RiskFactor（风险因素）：**必须提取具体的、细粒度的风险因素**，包括：
  * 具体的年龄段（如：高龄/年龄≥60岁）
  * 具体的疾病史（如：糖尿病、慢性肝病、慢性肾病，不要概括为"慢性疾病"）
  * 具体的生活习惯（如：酗酒、吸烟）
  * 具体的身体状况（如：肥胖、营养不良、免疫功能低下）
  * 具体的用药史（如：使用免疫抑制剂、长期使用激素）
  * 具体的其他因素（如：长期卧床、外伤史、手术史）
  **禁止**：不要将多个具体因素概括为笼统的类别（如将糖尿病、慢性肝病概括为"慢性疾病"）
- DifferentialDiagnosis（鉴别诊断）：需要鉴别的其他疾病
- LiteratureSource（文献来源）：专家共识、指南等文献

**关系类型定义：**
- HAS_SYMPTOM：疾病具有某种症状
- DIAGNOSED_BY：疾病通过某种方法诊断
- TREATED_WITH：疾病使用某种方法治疗
- CAUSED_BY：疾病由某种病原体引起
- HAS_RISK_FACTOR：疾病具有某种风险因素
- DIFFERENTIAL_DIAGNOSIS：疾病需要与某疾病鉴别
- SOURCE_FROM：实体来源于某文献

**提取要求：**
1. 仔细阅读文本，识别所有相关实体
2. **重要**：同一实体的中英文名称、缩写应识别为同一个实体，优先使用中文全称作为实体名称
   - 例如："坏死性软组织感染"、"NSTIs"、"Necrotizing soft tissue infections"是同一个疾病
   - 实体name字段使用中文全称，在description字段中必须注明所有别名："英文名：XXX，缩写：XXX"
3. **识别别名**：在整个文本中，无论出现中文全称、英文名称还是缩写，都必须识别为同一个实体
   - 例如：文中出现"NSTIs的症状包括..."，应识别"坏死性软组织感染"是source实体
   - 例如：文中出现"MRSA导致..."，应识别"耐甲氧西林金黄色葡萄球菌"是source实体
   - **关键**：在relationships中的source和target字段，无论原文使用什么名称，都必须统一使用中文全称
4. **别名映射常见示例**：
   - "坏死性软组织感染" = "NSTIs" = "Necrotizing soft tissue infections"
   - "耐甲氧西林金黄色葡萄球菌" = "MRSA" = "methicillin-resistant Staphylococcus aureus"
   - "A族溶血性链球菌" = "GAS" = "group A Streptococcus"
5. **风险因素提取要求**（重要！）：
   - 必须提取每一个具体的风险因素，不要合并或概括
   - 例如：看到"糖尿病、慢性肝病、慢性肾病"，应提取3个RiskFactor实体，而非1个"慢性疾病"
   - 例如：看到"高龄（年龄≥60岁）"，应提取为"高龄（年龄≥60岁）"，保留具体信息
   - 对于并列列举的风险因素，每个都要单独提取
   - **正确示例**：
     文本："易患人群包括高龄（年龄≥60岁）、酗酒、肥胖、糖尿病、慢性肝病、慢性肾病、使用免疫抑制剂"
     应提取7个RiskFactor：高龄（年龄≥60岁）、酗酒、肥胖、糖尿病、慢性肝病、慢性肾病、使用免疫抑制剂
   - **错误示例**：
     ❌ 将"糖尿病、慢性肝病、慢性肾病"概括为1个"慢性疾病"
     ❌ 将"使用免疫抑制剂或免疫功能低下"只提取1个，应提取2个独立因素
6. 为每个实体提供简洁准确的描述，必须在描述中包含所有出现过的别名
7. 识别实体之间的关系，确保关系类型正确
8. 保持专业术语的准确性
9. 输出必须是有效的JSON格式
"""),
    ("user", """请从以下医学文本中提取实体和关系：

{text}

请按照以下JSON格式输出：
{format_instructions}
""")
])

# 创建输出解析器
parser = JsonOutputParser(pydantic_object=KnowledgeGraph)

print("Prompt模板创建完成")


Prompt模板创建完成


## 5. 读取和分块处理文档


In [58]:
# 读取文档
doc_path = Path("ragt.md")
with open(doc_path, 'r', encoding='utf-8') as f:
    content = f.read()

print(f"文档总长度: {len(content)} 字符")

# 文档分块
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # 每块约2000字符
    chunk_overlap=200,  # 重叠200字符以保持上下文
    separators=["\n## ", "\n### ", "\n", " ", ""]
)

chunks = text_splitter.split_text(content)
print(f"文档已分成 {len(chunks)} 个块")

# 显示第一块示例
print("\n第一块内容示例：")
print(chunks[0][:300] + "...")


文档总长度: 11814 字符
文档已分成 7 个块

第一块内容示例：
## ·专家共识·

## 坏死性软组织感染临床诊治急诊专家共识

坏死性软组织感染(necrotizingsofttissueinfections，NSTIs)是由病原微生物感染导致皮下组织、筋膜或（和）肌肉坏死的一类疾病。NSTIs起病急、病情进展快，致死致残率高。为了进一步规范NSTIs的临床诊疗，中国医师协会急诊医师分会、中华医学会急诊医学分会、中国急诊专科医联体、中国研究型医院学会休克与脓毒症专业委员会、中国医师协会急救复苏和灾难医学专业委员会和浙江省医学会灾难医学分会组织我国急诊医学科、感染科、烧伤/创伤外科、创面修复科等学科的专家组成编写委员会，结合国内外最新进展和临床诊疗经验，...


## 6. 执行实体关系抽取


In [59]:
# 创建抽取链
extraction_chain = extraction_prompt | llm | parser

# 存储所有抽取结果
all_entities = []
all_relationships = []

# 对每个块进行抽取
print("开始实体关系抽取...")
start_time = time.time()

for i, chunk in enumerate(chunks):
    print(f"\n处理第 {i+1}/{len(chunks)} 块...")
    
    try:
        # 调用LLM进行抽取
        result = extraction_chain.invoke({
            "text": chunk,
            "format_instructions": parser.get_format_instructions()
        })
        
        # 收集结果
        if 'entities' in result:
            all_entities.extend(result['entities'])
            print(f"  提取了 {len(result['entities'])} 个实体")
        
        if 'relationships' in result:
            all_relationships.extend(result['relationships'])
            print(f"  提取了 {len(result['relationships'])} 个关系")
        
        # 避免请求过快
        time.sleep(1)
        
    except Exception as e:
        print(f"  处理第 {i+1} 块时出错: {str(e)}")
        continue

end_time = time.time()
print(f"\n抽取完成！总耗时: {end_time - start_time:.2f} 秒")
print(f"总共提取: {len(all_entities)} 个实体, {len(all_relationships)} 个关系")


开始实体关系抽取...

处理第 1/7 块...
  提取了 22 个实体
  提取了 10 个关系

处理第 2/7 块...
  提取了 38 个实体
  提取了 32 个关系

处理第 3/7 块...
  提取了 11 个实体
  提取了 10 个关系

处理第 4/7 块...
  提取了 38 个实体
  提取了 29 个关系

处理第 5/7 块...
  提取了 42 个实体
  提取了 39 个关系

处理第 6/7 块...
  提取了 56 个实体
  提取了 40 个关系

处理第 7/7 块...
  处理第 7 块时出错: Invalid json output: ```json
{
  "entities": [
    {
      "name": "坏死性软组织感染",
      "entity_type": "Disease",
      "description": "英文名：Necrotizing soft tissue infections，缩写：NSTIs"
    },
    {
      "name": "肿胀",
      "entity_type": "Symptom",
      "description": ""
    },
    {
      "name": "疼痛",
      "entity_type": "Symptom",
      "description": ""
    },
    {
      "name": "红斑",
      "entity_type": "Symptom",
      "description": ""
    },
    {
      "name": "发热",
      "entity_type": "Symptom",
      "description": ""
    },
    {
      "name": "血泡",
      "entity_type": "Symptom",
      "description": ""
    },
    {
      "name": "瘀斑",
      "entity_type": "Symptom",
      "description": ""
    

## 7. 去重和整理


In [60]:
# 实体去重（基于name和entity_type）
unique_entities = {}
for entity in all_entities:
    key = (entity['name'], entity['entity_type'])
    if key not in unique_entities:
        unique_entities[key] = entity
    else:
        # 如果描述更长，则更新
        if len(entity['description']) > len(unique_entities[key]['description']):
            unique_entities[key] = entity

unique_entities_list = list(unique_entities.values())

# 关系去重（基于source, target, relation_type）
unique_relationships = {}
for rel in all_relationships:
    key = (rel['source'], rel['target'], rel['relation_type'])
    if key not in unique_relationships:
        unique_relationships[key] = rel

unique_relationships_list = list(unique_relationships.values())

print(f"去重后: {len(unique_entities_list)} 个实体, {len(unique_relationships_list)} 个关系")

# 统计各类实体数量
entity_type_counts = {}
for entity in unique_entities_list:
    entity_type = entity['entity_type']
    entity_type_counts[entity_type] = entity_type_counts.get(entity_type, 0) + 1

print("\n各类实体数量：")
for entity_type, count in sorted(entity_type_counts.items()):
    print(f"  {entity_type}: {count}")

# 统计各类关系数量
relation_type_counts = {}
for rel in unique_relationships_list:
    relation_type = rel['relation_type']
    relation_type_counts[relation_type] = relation_type_counts.get(relation_type, 0) + 1

print("\n各类关系数量：")
for relation_type, count in sorted(relation_type_counts.items()):
    print(f"  {relation_type}: {count}")


去重后: 183 个实体, 148 个关系

各类实体数量：
  Disease: 27
  LiteratureSource: 13
  Pathogen: 7
  RiskFactor: 21
  Symptom: 43
  Test: 28
  Treatment: 44

各类关系数量：
  CAUSED_BY: 10
  DIAGNOSED_BY: 24
  DIFFERENTIAL_DIAGNOSIS: 7
  HAS_RISK_FACTOR: 17
  HAS_SYMPTOM: 39
  SOURCE_FROM: 12
  TREATED_WITH: 39


## 8. 保存结果


In [61]:
# 保存为JSON文件
output_data = {
    "entities": unique_entities_list,
    "relationships": unique_relationships_list,
    "metadata": {
        "source_document": "ragt.md",
        "extraction_time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "entity_count": len(unique_entities_list),
        "relationship_count": len(unique_relationships_list),
        "entity_type_counts": entity_type_counts,
        "relation_type_counts": relation_type_counts
    }
}

output_path = Path("extracted_knowledge_graph.json")
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"结果已保存到: {output_path.absolute()}")


结果已保存到: o:\MyProject\Knowleage\extracted_knowledge_graph.json


## 9. 查看抽取样例


In [62]:
# 显示部分实体样例
print("=== 实体样例 ===")
for entity_type in ['Disease', 'Symptom', 'Treatment']:
    print(f"\n{entity_type}类实体:")
    samples = [e for e in unique_entities_list if e['entity_type'] == entity_type][:3]
    for entity in samples:
        print(f"  - {entity['name']}: {entity['description'][:80]}...")

# 显示部分关系样例
print("\n=== 关系样例 ===")
for rel in unique_relationships_list[:10]:
    print(f"  ({rel['source']}) --[{rel['relation_type']}]--> ({rel['target']})")


=== 实体样例 ===

Disease类实体:
  - 坏死性软组织感染: 英文名：Necrotizing soft tissue infections，缩写：NSTIs...
  - 皮下组织（坏死性蜂窝织炎）: ...
  - 筋膜（坏死性筋膜炎）: ...

Symptom类实体:
  - 肿胀: ...
  - 疼痛: ...
  - 红斑: ...

Treatment类实体:
  - NSTIs快速响应小组: ...
  - 青霉素: ...
  - 克林霉素: ...

=== 关系样例 ===
  (坏死性软组织感染) --[CAUSED_BY]--> (A族溶血性链球菌)
  (坏死性软组织感染) --[CAUSED_BY]--> (耐甲氧西林金黄色葡萄球菌)
  (坏死性软组织感染) --[CAUSED_BY]--> (创伤弧菌)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (高龄（年龄≥60岁）)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (酗酒)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (肥胖)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (糖尿病)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (慢性肝病)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (慢性肾病)
  (坏死性软组织感染) --[HAS_RISK_FACTOR]--> (使用免疫抑制剂或免疫功能低下)


## 10. （可选）导入到Neo4j


In [63]:
# 如果需要导入到Neo4j，取消注释以下代码
 
from py2neo import Graph, Node, Relationship

# Neo4j连接配置
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "test1234"

# 连接到Neo4j
graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
print("已连接到Neo4j数据库")

# 创建节点映射
node_map = {}

# 创建实体节点
print("创建实体节点...")
for entity in unique_entities_list:
    node = Node(
        entity['entity_type'],
        name=entity['name'],
        description=entity['description']
    )
    graph.create(node)
    node_map[entity['name']] = node

print(f"已创建 {len(node_map)} 个节点")

# 创建关系
print("创建关系...")
created_relations = 0
for rel in unique_relationships_list:
    source_node = node_map.get(rel['source'])
    target_node = node_map.get(rel['target'])
    
    if source_node and target_node:
        relationship = Relationship(
            source_node,
            rel['relation_type'],
            target_node,
            description=rel.get('description', '')
        )
        graph.create(relationship)
        created_relations += 1

print(f"已创建 {created_relations} 个关系")
print("知识图谱已成功导入Neo4j！")
 

print("如需导入Neo4j，请取消注释上面的代码块")


已连接到Neo4j数据库
创建实体节点...
已创建 183 个节点
创建关系...
已创建 148 个关系
知识图谱已成功导入Neo4j！
如需导入Neo4j，请取消注释上面的代码块


## 总结

本notebook实现了基于GraphRAG思想的医学知识图谱自动抽取流程：

1. ✅ 使用指定的LLM配置（qwen2.5:14b）
2. ✅ 参考KG.ipynb中定义的实体和关系schema
3. ✅ 从ragt.md中自动提取结构化知识
4. ✅ 支持文档分块处理
5. ✅ 自动去重和统计
6. ✅ 保存为JSON格式
7. ✅ 可选导入到Neo4j数据库

**改进方向：**
- 增加实体消歧和链接
- 优化分块策略
- 添加质量评估机制
- 支持增量更新

**使用说明：**
1. 确保已安装依赖：`pip install langchain langchain-openai`
2. 确保ragt.md文件在同一目录下
3. 按顺序执行各个单元格
4. 查看生成的extracted_knowledge_graph.json文件
5. 如需导入Neo4j，取消注释最后一个代码块并配置连接信息
